# TomatoVision Pipeline 테스트

이미지 입력 → YOLO12 → FastSAM → 이진 마스크 이미지 출력 파이프라인을 단계별로 확인합니다.


In [ ]:
# 필요한 라이브러리 import
import autorootcwd
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO
from ultralytics import FastSAM

# 설정
image_path = "data/notebook_data/image.png"
yolo_model_path = "yolo_exp/yolo12n_laboro_exp3/weights/best.pt"
fastsam_model_path = "model/FastSAM-s.pt"
conf_thres = 0.25  # Validation과 동일한 설정 (predict_yolo.py 기본값)

# 디바이스 설정
device = 0 if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")


## 1단계: 모델 로드


In [ ]:
# YOLO 모델 로드
print(f"Loading YOLO model from: {yolo_model_path}")
yolo_model = YOLO(yolo_model_path)

# FastSAM 모델 로드
print(f"Loading FastSAM model from: {fastsam_model_path}")
fastsam_model = FastSAM(fastsam_model_path)

print("모델 로드 완료!")


## 2단계: 이미지 정보 확인


In [ ]:
# 이미지 로드
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"이미지 파일을 열 수 없습니다: {image_path}")

# 이미지 정보 가져오기
height, width = image.shape[:2]

print(f"이미지 정보:")
print(f"  - 경로: {image_path}")
print(f"  - 해상도: {width}x{height}")
print(f"  - 채널 수: {image.shape[2] if len(image.shape) == 3 else 1}")


## 3단계: 이미지 읽기 및 확인


In [ ]:
# BGR → RGB 변환
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 원본 이미지 확인
plt.figure(figsize=(12, 6))
plt.imshow(image_rgb)
plt.axis('off')
plt.title('원본 이미지')
plt.show()

print(f"이미지 크기: {image_rgb.shape}")


## 4단계: YOLO로 객체 감지


### 디버깅: 낮은 confidence threshold로 재시도 (선택사항)


In [ ]:
# YOLO로 객체 감지 (Validation과 동일한 설정)
# Validation에서는 conf=0.25를 사용하므로 동일하게 설정
print(f"Confidence threshold: {conf_thres} (Validation과 동일)")
print(f"이미지 경로: {image_path}")

# 여러 confidence 레벨로 테스트 (디버깅용)
print("\n" + "=" * 80)
print("다양한 Confidence Threshold로 테스트")
print("=" * 80)

test_conf_levels = [0.1, 0.15, 0.2, 0.25, 0.3, 0.5]
all_results = {}

for test_conf in test_conf_levels:
    # 파일 경로를 직접 전달하여 predict_yolo.py와 동일한 전처리 방식 사용
    test_results = yolo_model(image_path, conf=test_conf, imgsz=640, verbose=False)
    test_result = test_results[0]
    
    detection_count = len(test_result.boxes) if test_result.boxes is not None else 0
    tomato_count = 0
    max_conf = 0.0
    tomato_confs = []
    
    if test_result.boxes is not None and len(test_result.boxes) > 0:
        for box in test_result.boxes:
            cls_id = int(box.cls[0])
            cls_name = yolo_model.names[cls_id]
            conf = float(box.conf[0])
            max_conf = max(max_conf, conf)
            
            if cls_name in ['fully_ripened', 'half_ripened', 'green']:
                tomato_count += 1
                tomato_confs.append(conf)
    
    all_results[test_conf] = {
        'total': detection_count,
        'tomato': tomato_count,
        'max_conf': max_conf,
        'tomato_confs': tomato_confs,
        'result': test_result
    }
    
    conf_str = f", 토마토 conf: {[f'{c:.3f}' for c in tomato_confs[:3]]}" if tomato_confs else ""
    print(f"Conf={test_conf:.2f}: 전체 {detection_count}개, 토마토 {tomato_count}개, 최대 confidence: {max_conf:.3f}{conf_str}")

# Validation과 동일한 설정으로 최종 결과 사용
# 파일 경로를 직접 전달하여 predict_yolo.py와 동일한 전처리 방식 사용
# 이렇게 하면 YOLO가 파일에서 직접 로드하고 자체적으로 전처리하므로
# validation과 동일한 결과를 얻을 수 있습니다.
# imgsz=640으로 명시적으로 이미지 크기 지정
yolo_results = yolo_model(image_path, conf=conf_thres, imgsz=640, verbose=False)
yolo_result = yolo_results[0]

print("\n" + "=" * 80)
print(f"최종 결과 (conf={conf_thres}, Validation과 동일)")
print("=" * 80)

# 모델의 클래스 이름 확인
print("\n모델 클래스 정보:")
print(f"  총 클래스 수: {len(yolo_model.names)}")
for idx, name in yolo_model.names.items():
    print(f"    ID {idx}: {name}")

# 토마토 클래스만 필터링
tomato_class_names = ['fully_ripened', 'half_ripened', 'green']
tomato_indices = []
tomato_boxes = []
all_detections = []

# 모든 감지된 객체 확인 및 필터링
if yolo_result.boxes is not None and len(yolo_result.boxes) > 0:
    print(f"\n전체 감지된 객체 수: {len(yolo_result.boxes)}")
    print("\n[감지된 객체 목록 - 상세 정보]")
    
    # 이미지 크기
    img_height, img_width = image_rgb.shape[:2]
    
    for idx, box in enumerate(yolo_result.boxes):
        cls_id = int(box.cls[0])
        cls_name = yolo_model.names[cls_id]
        conf = float(box.conf[0])
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        
        # 박스 크기 계산
        box_width = x2 - x1
        box_height = y2 - y1
        box_area = box_width * box_height
        box_ratio = box_width / box_height if box_height > 0 else 0
        
        # 이미지 대비 박스 크기 비율
        area_ratio = box_area / (img_width * img_height)
        
        is_tomato = cls_name in tomato_class_names
        
        # 추가 필터링: 박스가 너무 크거나 비율이 이상하면 제외
        # 토마토는 일반적으로 작고 둥근 형태이므로, 너무 큰 박스나 비율이 극단적인 것은 제외
        is_valid_tomato = False
        if is_tomato:
            # 박스가 이미지의 50% 이상을 차지하면 제외 (벽 같은 큰 객체)
            # 박스 비율이 너무 극단적이면 제외 (너무 길거나 넓은 형태)
            if area_ratio < 0.5 and 0.2 < box_ratio < 5.0:
                is_valid_tomato = True
            else:
                print(f"  ⚠️ 잘못된 감지 제외: {cls_name} (신뢰도: {conf:.3f}, 크기: {area_ratio:.1%}, 비율: {box_ratio:.2f})")
        
        marker = "✓" if is_valid_tomato else ("✗" if is_tomato else "✗")
        print(f"  {marker} 객체 {idx+1}: {cls_name} (신뢰도: {conf:.3f}, 크기: {area_ratio:.1%}, 비율: {box_ratio:.2f}, 위치: ({int(x1)},{int(y1)})-({int(x2)},{int(y2)}))")
        
        all_detections.append({
            'idx': idx,
            'cls_name': cls_name,
            'conf': conf,
            'area_ratio': area_ratio,
            'box_ratio': box_ratio,
            'is_tomato': is_tomato,
            'is_valid': is_valid_tomato
        })
        
        if is_valid_tomato:
            tomato_indices.append(idx)
            tomato_boxes.append(box.xyxy[0])
    
    print(f"\n토마토로 필터링된 객체 수: {len(tomato_indices)} (전체 {len([d for d in all_detections if d['is_tomato']])}개 중)")
    
    # 토마토 박스 좌표 추출
    if len(tomato_boxes) > 0:
        yolo_box_coords = torch.stack(tomato_boxes)
    else:
        yolo_box_coords = None
else:
    print("\n⚠️ 감지된 객체가 없습니다!")
    yolo_box_coords = None

# 시각화 (토마토만 표시)
if len(tomato_indices) > 0:
    filtered_boxes = yolo_result.boxes[tomato_indices]
    yolo_annotated = yolo_result.plot(boxes=filtered_boxes)
    # YOLO plot() 결과가 BGR 형식일 수 있으므로 RGB로 변환
    if yolo_annotated.shape[2] == 3:
        yolo_annotated = cv2.cvtColor(yolo_annotated, cv2.COLOR_BGR2RGB)
    print("\n✓ 유효한 토마토 감지 성공!")
else:
    # 모든 감지 결과 표시 (디버깅용)
    if yolo_result.boxes is not None and len(yolo_result.boxes) > 0:
        yolo_annotated = yolo_result.plot()
        # YOLO plot() 결과가 BGR 형식일 수 있으므로 RGB로 변환
        if yolo_annotated.shape[2] == 3:
            yolo_annotated = cv2.cvtColor(yolo_annotated, cv2.COLOR_BGR2RGB)
        print("\n⚠️ 유효한 토마토는 감지되지 않았지만, 다른 객체는 감지되었습니다.")
        print("   (위의 상세 정보를 확인하여 잘못된 감지를 확인하세요)")
    else:
        yolo_annotated = image_rgb.copy()
        print("\n⚠️ 아무 객체도 감지되지 않았습니다.")

plt.figure(figsize=(12, 6))
plt.imshow(yolo_annotated)
plt.axis('off')
plt.title(f'YOLO 감지 결과 (conf=0.5, 유효한 토마토 {len(tomato_indices)}개)')
plt.show()


## 5단계: 토마토 클래스만 필터링


In [ ]:
# 토마토가 감지된 경우에만 FastSAM 수행
if yolo_box_coords is not None and len(yolo_box_coords) > 0:
    print("FastSAM 세그멘테이션 수행 중...")
    
    fastsam_results = fastsam_model.predict(
        source=image_rgb,
        bboxes=yolo_box_coords,
        device=device,
        retina_masks=True,
        imgsz=640,
        conf=0.5,
        iou=0.9,
        verbose=False
    )
    
    # FastSAM 결과 시각화
    annotated_image = fastsam_results[0].plot()

    plt.figure(figsize=(12, 6))
    plt.imshow(annotated_image)
    plt.axis('off')
    plt.title('FastSAM 세그멘테이션 결과')
    plt.show()
    
    print("FastSAM 세그멘테이션 완료!")
else:
    print("토마토가 감지되지 않아 FastSAM을 건너뜁니다.")
    fastsam_results = None


In [ ]:
yolo_box_coords

## 6단계: FastSAM으로 세그멘테이션 (토마토가 감지된 경우)


In [ ]:
# 토마토가 감지된 경우에만 FastSAM 수행
if yolo_box_coords is not None and len(yolo_box_coords) > 0:
    print("FastSAM 세그멘테이션 수행 중...")
    
    fastsam_results = fastsam_model.predict(
        source=image_rgb,
        bboxes=yolo_box_coords,
        device=device,
        retina_masks=True,
        imgsz=640,
        conf=0.5,
        iou=0.9,
        verbose=False
    )
    
    # FastSAM 결과 시각화
    annotated_image = fastsam_results[0].plot()
    
    plt.figure(figsize=(12, 6))
    plt.imshow(annotated_image)
    plt.axis('off')
    plt.title('FastSAM 세그멘테이션 결과')
    plt.show()
    
    print("FastSAM 세그멘테이션 완료!")
else:
    print("토마토가 감지되지 않아 FastSAM을 건너뜁니다.")
    fastsam_results = None


## 7단계: 마스크 합치기 및 이진 마스크 생성


In [ ]:
if fastsam_results is not None:
    # 마스크 합치기 (OR 연산)
    masks = fastsam_results[0].masks.data.cpu().numpy()
    
    print(f"마스크 shape: {masks.shape}")
    
    if len(masks.shape) >= 2:
        # 여러 마스크가 있는 경우 합치기
        binary_mask = np.any(masks > 0, axis=0)
        print(f"합쳐진 마스크 shape: {binary_mask.shape}")
    else:
        binary_mask = masks
    
    # 이진 마스크를 uint8로 변환 (0 또는 255)
    binary_image = (binary_mask * 255).astype(np.uint8)
    
    # 원본 프레임 크기로 리사이즈 (필요한 경우)
    if binary_image.shape != (height, width):
        binary_image = cv2.resize(binary_image, (width, height), interpolation=cv2.INTER_NEAREST)
        print(f"마스크 리사이즈: {binary_image.shape}")
    
    # 이진 마스크 시각화
    plt.figure(figsize=(12, 6))
    plt.imshow(binary_image, cmap='gray')
    plt.axis('off')
    plt.title('이진 마스크 (토마토 영역)')
    plt.show()
    
    print("이진 마스크 생성 완료!")
else:
    print("FastSAM 결과가 없어 이진 마스크를 생성할 수 없습니다.")
    binary_image = np.zeros((height, width), dtype=np.uint8)


## 8단계: 최종 결과 비교


In [ ]:
# 모든 결과를 한 번에 비교
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 원본 이미지
axes[0, 0].imshow(image_rgb)
axes[0, 0].set_title('1. 원본 이미지')
axes[0, 0].axis('off')

# YOLO 결과
axes[0, 1].imshow(yolo_annotated)
axes[0, 1].set_title('2. YOLO 감지 결과 (토마토만)')
axes[0, 1].axis('off')

# FastSAM 결과 (있는 경우)
if fastsam_results is not None:
    annotated_image = fastsam_results[0].plot()
    axes[1, 0].imshow(annotated_image)
    axes[1, 0].set_title('3. FastSAM 세그멘테이션')
else:
    axes[1, 0].imshow(image_rgb)
    axes[1, 0].set_title('3. FastSAM 세그멘테이션 (토마토 미감지)')
axes[1, 0].axis('off')

# 이진 마스크
axes[1, 1].imshow(binary_image, cmap='gray')
axes[1, 1].set_title('4. 이진 마스크 (최종 출력)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()
